# Crawling

web crawler: 웹 페이지의 데이터를 모아주는 소프트웨어</br>
web crawling : 크롤러를 사용해 웹 페이지의 데이터를 추출해 내는 행위

## requests

requests는 python용 HTTP 라이브러리</br>
Crawling 과정에서 requests모듈을 이용해 웹 사이트의 소스코드를 가져온 다음 파싱을 하는 경우가 많다

HTTP 요청 메서드(HTTP request method): 클라이언트가 서버에게 요청하는 목적 및 그 종류를 알리는 수단 (request을 보내면 response를 객체로 받는다)<br>
이 chapter에서는 crawling이 목적이기 때문에, 데이터 요청 외 메서드(put, head, patch, delete, options)는 다루지 않는다
- **get**: requests.get(url, params, **kwargs)
    - params: HTTP 요청을 할 때 쿼리 스트링(query string)을 통해 응답받을 데이터를 필터링 (url에서 추출하고자 하는 class와 값 지정)
    ```python
        test_response = requests.get("https://jsonplaceholder.typicode.com/posts", params = {'user_Id': 1})
        test_response.json()
        """
        [{'userId': 1,
          'id': 1,
          'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit',
          'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'}]
        """
    ```
- **post**: requests.post(url, data=None, json=None, **kwargs)
    - data: get의 params와 같은 역할
    - json: HTTP 요청 바디에 JSON데이터를 전송하기 위한 인자
- status_code: 온라인 서비스를 HTTP로 호출하면 상태를 응답받고, 이 상태를 통해 요청이 잘 처리됐는지 판단한다
    - 200: 정상
    - 404: 페이지를 찾을 수 없음
---
```plain text
GET vs POST

get 메서드는 URL에 파라미터를 추가하여 서버에 GET 요청을 보내는 함수이고, post 메서드는 서버에 POST 요청(주로 서버에 데이터를 제출하기 위해 사용)을 보내는 함수이다

GET 요청은 데이터를 전송하는 바디가 필요하지 않고, URL에 파라미터를 추가하여 전송한다
GET 요청은 주로 웹 브라우저에서 링크를 클릭하여 페이지를 요청하거나, 검색어를 입력하여 검색 결과를 요청할 때 사용한다
GET 요청은 캐시를 사용하여 이미 조회된 데이터를 재사용할 수 있어서 반복적인 요청에서 유리하다

POST 요청은 데이터를 전송하는 바디가 필요하며, 데이터는 요청 바디에 추가된다
POST 요청은 주로 로그인 페이지에서 사용자 이름과 비밀번호를 입력하고 제출할 때, 또는 회원 가입 페이지에서 회원 정보를 입력하고 제출할 때 사용한다
POST 요청은 요청 바디에 보안이 필요한 데이터를 추가할 수 있어서 보안이 중요한 작업에서 사용한다
```
---
HTTP 요청의 응답으로 가져온 웹 사이트의 소스코드를 출력하는 형태
- text
- content (binary)
- json()
- headers: 응답에 대한 메타 데이터를 담고 있는 응답 헤더를 딕셔너리 형태로 반환

## BeautifulSoup

HTML과 XML파일에서 데이터를 추출하는 라이브러리

**parsing**

>언어학에서 parsing은 구문 분석이라고도하며 문장을 그것을 이루고 있는 구성 성분으로 분해하고 그들 사이의 위계 관계를 분석하여 문장의 구조를 결정하는 것이다

parsing은 일련의 문자열을 의미있는 token(어휘 분석의 단위) 으로 분해하고 그것들로 이루어진 Parse tree를 만드는 과정으로,</br>

***데이터를 조립해 원하는 형태로 만드는 것을 의미***한다

**BeautifulSoup의 기본 구조**
```python
BeautifulSoup(markup, features, from_encoding, exclude_encodings, **kwargs)
```
- markup: HTML을 구성하는 언어로, 클라이언트의 요청에 대한 서버의 응답으로 생성된 requests객체를 입력한다
- features: parser 지정, 'lxml', 'lxml-xml', 'html.parser', 'html5lib', 'html', 'xml' 등
- from_encoding: 인코딩 종류 지정
- exclude_encodings: 제외할 인코딩 지정, 인코딩 타입을 정확히 할지는 못할 때 확실히 아닌 인코딩을 지정해서 제외한다
- \*\*kwargs: Beautiful Soup 4 사용자를 위해 변경된 명령어를 알려준다 ('convertEntities', 'markupMessage' 등)

### BeautifulSoup method

requests객체를 parsing한 BeautifulSoup객체에서 태그를 추출하려면 메서드를 이용해야 한다

- find(name, attrs, recursive, string, \*\*kwargs): 태그 추출 (chaining을 통해 하위 태그로 접근할 수 있다)
- find_all(name, attrs, recursive, string, \*\*kwargs): 지정 태그 모두 추출
- select(): 지정한 규칙에 일치하는 태그들을 모두 반환
    - tag: 지정한 tag를 반환한다
    - tag.class: 지정한 class명을 가진 tag를 반환한다
    - \>: 태그의 상하위 관계를 표현한다
    - [attr_name="attr"]: 지정한 속성을 가진 tag를 반환한다
        - 속성의 값이 특정 값과 정확히 일치하는 경우: =
        - 특정 값으로 시작하는 경우: ^=
        - 특정 값으로 끝나는 경우: $=
        - 특정 값을 포함하는 경우: *=
        - 특정 값을 포함하지 않는 경우: !=
- select_one(): 지정한 규칙에 일치하는 tag 중 가장 먼저 만난 tag 한 개를 반환

https://www.crummy.com/software/BeautifulSoup/bs4/doc.ko/

## 동적 페이지

**정적 페이지**는 인터넷에 접속하면 페이지의 모든 정보가 업로드 된다</br>
html을 불러오면 추가 작업 없이 그 안에 모든 데이터를 추출할 수 있다

**동적 페이지**는 페이지 내에서 사용자가 특정한 동작을 하면 추가적인 정보가 업데이트 된다. 단, 이 때 url이 변경되지 않는다</br>
(특정 동작은 클릭, 스크롤, 로그인 등의 동작을 말한다)

ex. 네이버 다음 등 웹사이트의 배너, 인기 검색어, SNS 피드 등

동적 페이지에서 데이터를 추출하기 위해서는 추출하고자 하는 데이터의 html이 갱신되는데 필요한 동작을 실행시켜줘야 한다</br>
파이썬을 통해 동작을 자동으로 실행 시키기 위해서는 2가지, 동작 자동화 주문 셀레니움과 웹 제어도구 웹 드라이버가 필요하다 (셀리네움으로 동작을 지시하면 웹 드라이버에서 동작을 수행하는 것)

이 chapter에서 셀레니움은 다루지 않는다

https://charimlab.tistory.com/entry/ep01%EC%9B%B9%ED%81%AC%EB%A1%A4%EB%A7%81-11-%EB%8F%99%EC%A0%81-%ED%8E%98%EC%9D%B4%EC%A7%80%EC%9B%B9-%EB%8F%99%EC%9E%91-%EC%9E%90%EB%8F%99%ED%99%94Selenium-with-%ED%8C%8C%EC%9D%B4%EC%8D%AC

## 네이버 홈페이지에서 웹툰 배너의 인기급상승 웹툰 크롤링

네이버는 웹툰 배너를 클릭하면 화면에 출력되는 정보는 업데이트 되지만, URL은 변하지 않는 동적 페이지이다

아래 코드는 해당 배너의 URL에 request를 보내 crawling하는 예시이다

In [344]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.naver.com/nvhaproxy/v1/panels/BBOOM/html'
headers = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
   ,'referer':'https://datalab.naver.com/'
}

response = requests.get(url)

if response.status_code == requests.codes.ok:
    print('접속성공')
    soup = BeautifulSoup(response.text, 'html.parser')

접속성공


In [284]:
response

<Response [200]>

In [208]:
soup

<div class="group_topstory" data-block-code="PC-THEME-BBOOM-EDIT-AREA" data-block-id="645cb11f5cea28d2db1df63b" data-block-type="BLOCKS" data-da="margin-top" data-template-code="PC-THEMECAST-EDIT-AREA">
<div class="topstory_inner" data-block-code="PC-THEME-BBOOM-EDIT-AREA-ITEM" data-block-id="645ca8d97dd28a7aefb4c12d" data-block-type="A-MATERIAL" data-template-code="IMAGE1">
<div class="topstory_view">
<a class="topstory_thumb" data-clk="tcc_web.editbigimg1" href="https://comic.naver.com/webtoon/detail?titleId=783540&amp;no=58" target="_blank">
<img alt="휴재 복귀 &lt;여름여자 하보이&gt; 최신화 보러가기" data-src="https://s.pstatic.net/static/www/mobile/edit/20230511/cropImg_728x360_125734535543999606.png" height="180" onerror="this.outerHTML='&lt;span class=\'pm_noimg\'&gt;&lt;span class=\'noimg_text\'&gt;이미지 준비중&lt;/span&gt;&lt;/span&gt;'" src="https://s.pstatic.net/static/www/mobile/edit/20230511/cropImg_728x360_125734535543999606.png" width="364"/>
<span class="thumb_bd"></span>
</a>
<a class="topst

In [343]:
results = soup.select('a.chart_area[data-clk^="tcc_web.hottab1cont"]>div.chart_info>div.info_box>strong.title')

lst = []

for result in results:
    lst.append(result.get_text())
        
print(lst)

['외모지상주의', '역대급 영지 설계사', '광마회귀', '나 혼자 만렙 뉴비', '죽지 않으려면', '재혼 황후', '어쩌다보니 천생연분', '1초', '나노마신', '언니, 이번 생엔 내가 왕비야']


### 네이버 url에서 crawling 예시 코드

In [351]:
url = "https://www.naver.com/"
while 1:
    response = requests.get(url)
    data = BeautifulSoup(response.text, 'html.parser')
    try:
        if re.findall(r'true', str(data.find('a', {"class":"_NM_THEME_CATE tab id_bboom"})))[0] == 'true':
            break
    except IndexError:
        pass

toon_top_10 = data.findAll("div", {"class" : "chart_view_wrap type_webtoon"})[0]
toon_lst = toon_top_10.findAll("strong", "title")
toon_rank = ["{0}. ".format(i+1) + toon_lst[i].get_text() for i in range(len(toon_lst))]   
toon_rank

['1. 외모지상주의',
 '2. 역대급 영지 설계사',
 '3. 광마회귀',
 '4. 나 혼자 만렙 뉴비',
 '5. 재혼 황후',
 '6. 택배기사',
 '7. 죽지 않으려면',
 '8. 어쩌다보니 천생연분',
 '9. 언니, 이번 생엔 내가 왕비야',
 '10. 물위의 우리']

## 네이버 실시간 급상승 검색어 크롤링

크롤링 막은 사이트 접속 권한(headers) 설정</br>
https://luminitworld.tistory.com/88

In [307]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import json

headers = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
   ,'referer':'https://datalab.naver.com/'
}
url = "https://datalab.naver.com/shoppingInsight/getKeywordRank.naver?timeUnit=date&cid=50000001"

response = requests.post(url, headers = headers)

if response.status_code == requests.codes.ok:
    print('접속성공')

접속성공


In [305]:
# json 형태
response.text[:500]

'[{"message":null,"statusCode":200,"returnCode":0,"date":"2023/04/29","datetime":"2023.04.29.(토)","range":"","ranks":[{"rank":1,"keyword":"크록스","linkId":"크록스"},{"rank":2,"keyword":"나이키운동화","linkId":"나이키운동화"},{"rank":3,"keyword":"캐리어","linkId":"캐리어"},{"rank":4,"keyword":"운동화","linkId":"운동화"},{"rank":5,"keyword":"에코백","linkId":"에코백"},{"rank":6,"keyword":"거꾸로우산","linkId":"거꾸로우산"},{"rank":7,"keyword":"크로스백","linkId":"크로스백"},{"rank":8,"keyword":"안전화","linkId":"안전화"},{"rank":9,"keyword":"레인부츠","linkId"'

In [341]:
# json 형태로 변경
data = json.loads(response.text)
data[0]

{'message': None,
 'statusCode': 200,
 'returnCode': 0,
 'date': '2023/04/29',
 'datetime': '2023.04.29.(토)',
 'range': '',
 'ranks': [{'rank': 1, 'keyword': '크록스', 'linkId': '크록스'},
  {'rank': 2, 'keyword': '나이키운동화', 'linkId': '나이키운동화'},
  {'rank': 3, 'keyword': '캐리어', 'linkId': '캐리어'},
  {'rank': 4, 'keyword': '운동화', 'linkId': '운동화'},
  {'rank': 5, 'keyword': '에코백', 'linkId': '에코백'},
  {'rank': 6, 'keyword': '거꾸로우산', 'linkId': '거꾸로우산'},
  {'rank': 7, 'keyword': '크로스백', 'linkId': '크로스백'},
  {'rank': 8, 'keyword': '안전화', 'linkId': '안전화'},
  {'rank': 9, 'keyword': '레인부츠', 'linkId': '레인부츠'},
  {'rank': 10, 'keyword': '루이비통가방', 'linkId': '루이비통가방'}]}

In [340]:
for i in data:
    print(datetime.strptime(i['datetime'][:-4], '%Y.%m.%d').strftime('%Y년 %m월 %d일의 패션잡화 검색어 순위입니다.\n'))

    for rank in i['ranks']:
        print(f"{rank['rank']}위 {rank['keyword']}")
    print()

2023년 04월 29일의 패션잡화 검색어 순위입니다.

1위 크록스
2위 나이키운동화
3위 캐리어
4위 운동화
5위 에코백
6위 거꾸로우산
7위 크로스백
8위 안전화
9위 레인부츠
10위 루이비통가방

2023년 04월 30일의 패션잡화 검색어 순위입니다.

1위 크록스
2위 나이키운동화
3위 캐리어
4위 운동화
5위 크로스백
6위 에코백
7위 슬리퍼
8위 아디다스운동화
9위 백팩
10위 안전화

2023년 05월 01일의 패션잡화 검색어 순위입니다.

1위 크록스
2위 나이키운동화
3위 캐리어
4위 운동화
5위 레인부츠
6위 크로스백
7위 에코백
8위 아디다스운동화
9위 슬리퍼
10위 지비츠

2023년 05월 02일의 패션잡화 검색어 순위입니다.

1위 크록스
2위 나이키운동화
3위 캐리어
4위 레인부츠
5위 운동화
6위 에코백
7위 슬리퍼
8위 크로스백
9위 카드지갑
10위 지비츠

2023년 05월 03일의 패션잡화 검색어 순위입니다.

1위 크록스
2위 레인부츠
3위 나이키운동화
4위 캐리어
5위 운동화
6위 슬리퍼
7위 에코백
8위 크로스백
9위 안전화
10위 양산

2023년 05월 04일의 패션잡화 검색어 순위입니다.

1위 크록스
2위 레인부츠
3위 나이키운동화
4위 헌터레인부츠
5위 캐리어
6위 운동화
7위 슬리퍼
8위 에코백
9위 크로스백
10위 장화

2023년 05월 05일의 패션잡화 검색어 순위입니다.

1위 레인부츠
2위 크록스
3위 헌터레인부츠
4위 아디다스노라
5위 나이키운동화
6위 락피쉬레인부츠
7위 장화
8위 캐리어
9위 운동화
10위 우산

2023년 05월 06일의 패션잡화 검색어 순위입니다.

1위 레인부츠
2위 크록스
3위 헌터레인부츠
4위 아디다스노라
5위 락피쉬레인부츠
6위 장화
7위 나이키운동화
8위 캐리어
9위 운동화
10위 우산

2023년 05월 07일의 패션잡화 검색어 순위입니다.

1위 크록스
2위 레인부츠
3위 나이키운동화
4위 헌터레인부츠
5위 락피쉬레인부츠
6위 캐리어
7위 운동화
8위 장화
9위 